In [1]:
import urllib
import ntpath
from pipelineapi import PipelineAPI
import os
import glob


def path_leaf(path):
    if urllib.parse.urlparse(path).scheme != "" or os.path.isabs(path):
        path = os.path.split(path)[-1]
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

In [2]:
from pipelineapi import PipelineAPI
import os

def dataset_import(dataset_name, dataset_file):
        data_pipeline_api = PipelineAPI("http://ec2-54-191-65-138.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')
        with open(dataset_file, 'rb') as f:
            data_pipeline_api.upload_json_file(f, file_name=path_leaf(dataset_file), round='inspection', dataset_name=dataset_name, skip_image=False)
            

In [3]:
def update_cats(data):
    for img_data in data:
        if img_data.get('labels', None):
            deletions = []
            for idx, label in enumerate(img_data['labels']):
                # If detected - delete
                if 'detected-' in label['category']:
                    label['category'] = label['category'].replace('detected-', '')
                if 'tracked-' in label['category']:
                    deletions.append(label)
                if 'flickering_' in label['category']:
                    label['category'] = label['category'].replace('flickering_', '').replace('tracked-', '')
            
            for label in deletions:
                img_data['labels'].remove(label)
                
    return data

In [4]:
def strip_kache_data(data):
    for img_data in data:
        kache_id = None
        if img_data.get('labels', None):
            for idx, label in enumerate(img_data['labels']):
                    label['kache_label_id'] = None
    return data

In [7]:
import glob
import json

for dset in glob.glob('/media/kuser/Data/080519_trn/*.json'):
    with open(dset, 'r') as fn:
        data = json.load(fn)

    data = update_cats(data)
    # Strip flicker mining database indexes
    data = strip_kache_data(data)
    
    with open(dset, 'w+') as fn:
        json.dump(data, fn)

KeyboardInterrupt: 

In [ ]:
import urllib
import ntpath
import json
import os

def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['062019', '032019']
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[10], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#           anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--inspected.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--inspected.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
import urllib
import ntpath
import json
import os

def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-149-236-75.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = ['062019', '032019']
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=['motor','motorbike','motorcycle'],
                    label_statuses=[], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#           anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 25000 == 0:
            with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--motorcycles.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{"^".join(dataset_names)}_{batch}--motorcycles.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
get_dataset()

In [ ]:
def get_dataset():
    data_pipeline_api = PipelineAPI("http://ec2-54-191-65-138.us-west-2.compute.amazonaws.com/", 'admin@pronto.ai', 'admin123!')

    batch_size = 0
    batch = 0
    batch_anns = []


    dataset_names = 'china_set'
    kache_anns_dir =  '/home/dean/Desktop'

    databunch = data_pipeline_api.data(dataset_names=dataset_names, video_source_names=[], categories_names=[],
                    label_statuses=[20], page_size=None, frame_id=None) # returns generator

    for anns in databunch:
#             anns = update_traffic_light_cats([anns])
        batch_anns.append(anns)
        batch_size += len(anns)
        if batch_size % 5000 == 0:
            with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
                json.dump(batch_anns, fn)
            batch_anns = []
            batch += 1

    # Save any remaining frames
    if len(batch_anns) != 0:
        with open(os.path.join(kache_anns_dir,f'{dataset_names}_{batch}--rejected.json'), 'w') as fn:
            json.dump(batch_anns, fn)


In [ ]:
get_dataset()

In [13]:
from collections import defaultdict
import numpy as np

attr2idx = {}
idx2attr = {}

In [14]:
attr2idx['video_source'] = 0
attr2idx['cam_num'] = 1
attr2idx['latitude'] = 2
attr2idx['longitude'] = 3
attr2idx['hour_of_day'] = 4

# Create vector that stores a representation of the object distribution from the log, sorted by class, then type, then box size (in descending order)
classes = ["bike", "bus", "car", "construct-equipment", "construct-cone","construct-pole", "construct-post", "construct-barrel","construct-sign", "motorcycle", "person", "rider", "traffic light", "traffic light-amber", "traffic light-green", "traffic light-red", "traffic sign", "traffic sign-slow_sign", "traffic sign-speed_sign", "traffic sign-stop_sign", "trailer", "train", "truck",]
#
idx = 5
for cls in classes:
    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:
        print(f'total_{tracking_type}-{cls}',':',idx)
        attr2idx[f'total_{tracking_type}-{cls}'] = idx
        idx += 1

for cls in classes:
    for tracking_type in ['detected', 'tracked', 'flickering_tracked']:
        # Load the first 10 objects in each category
        for i in np.arange(1, 11):
            
            #x1 
            print(f'{tracking_type}-{cls}_{i}_x1',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_x1'] = idx
            idx += 1
            
            #y1 
            print(f'{tracking_type}-{cls}_{i}_y1',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_y1'] = idx
            idx += 1
            
            #x2 
            print(f'{tracking_type}-{cls}_{i}_x2',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_x2'] = idx
            idx += 1
            
            #y2 
            print(f'{tracking_type}-{cls}_{i}_y2',':',idx)
            attr2idx[f'{tracking_type}-{cls}_{i}_y2'] = idx
            idx += 1

idx2attr = {v_i: v for v, v_i in attr2idx.items()}          
    

total_detected-bike : 5
total_tracked-bike : 6
total_flickering_tracked-bike : 7
total_detected-bus : 8
total_tracked-bus : 9
total_flickering_tracked-bus : 10
total_detected-car : 11
total_tracked-car : 12
total_flickering_tracked-car : 13
total_detected-construct-equipment : 14
total_tracked-construct-equipment : 15
total_flickering_tracked-construct-equipment : 16
total_detected-construct-cone : 17
total_tracked-construct-cone : 18
total_flickering_tracked-construct-cone : 19
total_detected-construct-pole : 20
total_tracked-construct-pole : 21
total_flickering_tracked-construct-pole : 22
total_detected-construct-post : 23
total_tracked-construct-post : 24
total_flickering_tracked-construct-post : 25
total_detected-construct-barrel : 26
total_tracked-construct-barrel : 27
total_flickering_tracked-construct-barrel : 28
total_detected-construct-sign : 29
total_tracked-construct-sign : 30
total_flickering_tracked-construct-sign : 31
total_detected-motorcycle : 32
total_tracked-motorcycl

tracked-construct-cone_10_y1 : 631
tracked-construct-cone_10_x2 : 632
tracked-construct-cone_10_y2 : 633
flickering_tracked-construct-cone_1_x1 : 634
flickering_tracked-construct-cone_1_y1 : 635
flickering_tracked-construct-cone_1_x2 : 636
flickering_tracked-construct-cone_1_y2 : 637
flickering_tracked-construct-cone_2_x1 : 638
flickering_tracked-construct-cone_2_y1 : 639
flickering_tracked-construct-cone_2_x2 : 640
flickering_tracked-construct-cone_2_y2 : 641
flickering_tracked-construct-cone_3_x1 : 642
flickering_tracked-construct-cone_3_y1 : 643
flickering_tracked-construct-cone_3_x2 : 644
flickering_tracked-construct-cone_3_y2 : 645
flickering_tracked-construct-cone_4_x1 : 646
flickering_tracked-construct-cone_4_y1 : 647
flickering_tracked-construct-cone_4_x2 : 648
flickering_tracked-construct-cone_4_y2 : 649
flickering_tracked-construct-cone_5_x1 : 650
flickering_tracked-construct-cone_5_y1 : 651
flickering_tracked-construct-cone_5_x2 : 652
flickering_tracked-construct-cone_5_y2 :

flickering_tracked-construct-sign_5_y1 : 1131
flickering_tracked-construct-sign_5_x2 : 1132
flickering_tracked-construct-sign_5_y2 : 1133
flickering_tracked-construct-sign_6_x1 : 1134
flickering_tracked-construct-sign_6_y1 : 1135
flickering_tracked-construct-sign_6_x2 : 1136
flickering_tracked-construct-sign_6_y2 : 1137
flickering_tracked-construct-sign_7_x1 : 1138
flickering_tracked-construct-sign_7_y1 : 1139
flickering_tracked-construct-sign_7_x2 : 1140
flickering_tracked-construct-sign_7_y2 : 1141
flickering_tracked-construct-sign_8_x1 : 1142
flickering_tracked-construct-sign_8_y1 : 1143
flickering_tracked-construct-sign_8_x2 : 1144
flickering_tracked-construct-sign_8_y2 : 1145
flickering_tracked-construct-sign_9_x1 : 1146
flickering_tracked-construct-sign_9_y1 : 1147
flickering_tracked-construct-sign_9_x2 : 1148
flickering_tracked-construct-sign_9_y2 : 1149
flickering_tracked-construct-sign_10_x1 : 1150
flickering_tracked-construct-sign_10_y1 : 1151
flickering_tracked-construct-sig

flickering_tracked-traffic light_10_y1 : 1631
flickering_tracked-traffic light_10_x2 : 1632
flickering_tracked-traffic light_10_y2 : 1633
detected-traffic light-amber_1_x1 : 1634
detected-traffic light-amber_1_y1 : 1635
detected-traffic light-amber_1_x2 : 1636
detected-traffic light-amber_1_y2 : 1637
detected-traffic light-amber_2_x1 : 1638
detected-traffic light-amber_2_y1 : 1639
detected-traffic light-amber_2_x2 : 1640
detected-traffic light-amber_2_y2 : 1641
detected-traffic light-amber_3_x1 : 1642
detected-traffic light-amber_3_y1 : 1643
detected-traffic light-amber_3_x2 : 1644
detected-traffic light-amber_3_y2 : 1645
detected-traffic light-amber_4_x1 : 1646
detected-traffic light-amber_4_y1 : 1647
detected-traffic light-amber_4_x2 : 1648
detected-traffic light-amber_4_y2 : 1649
detected-traffic light-amber_5_x1 : 1650
detected-traffic light-amber_5_y1 : 1651
detected-traffic light-amber_5_x2 : 1652
detected-traffic light-amber_5_y2 : 1653
detected-traffic light-amber_6_x1 : 1654
d

detected-traffic sign-slow_sign_5_y1 : 2131
detected-traffic sign-slow_sign_5_x2 : 2132
detected-traffic sign-slow_sign_5_y2 : 2133
detected-traffic sign-slow_sign_6_x1 : 2134
detected-traffic sign-slow_sign_6_y1 : 2135
detected-traffic sign-slow_sign_6_x2 : 2136
detected-traffic sign-slow_sign_6_y2 : 2137
detected-traffic sign-slow_sign_7_x1 : 2138
detected-traffic sign-slow_sign_7_y1 : 2139
detected-traffic sign-slow_sign_7_x2 : 2140
detected-traffic sign-slow_sign_7_y2 : 2141
detected-traffic sign-slow_sign_8_x1 : 2142
detected-traffic sign-slow_sign_8_y1 : 2143
detected-traffic sign-slow_sign_8_x2 : 2144
detected-traffic sign-slow_sign_8_y2 : 2145
detected-traffic sign-slow_sign_9_x1 : 2146
detected-traffic sign-slow_sign_9_y1 : 2147
detected-traffic sign-slow_sign_9_x2 : 2148
detected-traffic sign-slow_sign_9_y2 : 2149
detected-traffic sign-slow_sign_10_x1 : 2150
detected-traffic sign-slow_sign_10_y1 : 2151
detected-traffic sign-slow_sign_10_x2 : 2152
detected-traffic sign-slow_si

detected-train_10_y1 : 2631
detected-train_10_x2 : 2632
detected-train_10_y2 : 2633
tracked-train_1_x1 : 2634
tracked-train_1_y1 : 2635
tracked-train_1_x2 : 2636
tracked-train_1_y2 : 2637
tracked-train_2_x1 : 2638
tracked-train_2_y1 : 2639
tracked-train_2_x2 : 2640
tracked-train_2_y2 : 2641
tracked-train_3_x1 : 2642
tracked-train_3_y1 : 2643
tracked-train_3_x2 : 2644
tracked-train_3_y2 : 2645
tracked-train_4_x1 : 2646
tracked-train_4_y1 : 2647
tracked-train_4_x2 : 2648
tracked-train_4_y2 : 2649
tracked-train_5_x1 : 2650
tracked-train_5_y1 : 2651
tracked-train_5_x2 : 2652
tracked-train_5_y2 : 2653
tracked-train_6_x1 : 2654
tracked-train_6_y1 : 2655
tracked-train_6_x2 : 2656
tracked-train_6_y2 : 2657
tracked-train_7_x1 : 2658
tracked-train_7_y1 : 2659
tracked-train_7_x2 : 2660
tracked-train_7_y2 : 2661
tracked-train_8_x1 : 2662
tracked-train_8_y1 : 2663
tracked-train_8_x2 : 2664
tracked-train_8_y2 : 2665
tracked-train_9_x1 : 2666
tracked-train_9_y1 : 2667
tracked-train_9_x2 : 2668
tracke

In [15]:
print(idx2attr)

{0: 'video_source', 1: 'cam_num', 2: 'latitude', 3: 'longitude', 4: 'hour_of_day', 5: 'total_detected-bike', 6: 'total_tracked-bike', 7: 'total_flickering_tracked-bike', 8: 'total_detected-bus', 9: 'total_tracked-bus', 10: 'total_flickering_tracked-bus', 11: 'total_detected-car', 12: 'total_tracked-car', 13: 'total_flickering_tracked-car', 14: 'total_detected-construct-equipment', 15: 'total_tracked-construct-equipment', 16: 'total_flickering_tracked-construct-equipment', 17: 'total_detected-construct-cone', 18: 'total_tracked-construct-cone', 19: 'total_flickering_tracked-construct-cone', 20: 'total_detected-construct-pole', 21: 'total_tracked-construct-pole', 22: 'total_flickering_tracked-construct-pole', 23: 'total_detected-construct-post', 24: 'total_tracked-construct-post', 25: 'total_flickering_tracked-construct-post', 26: 'total_detected-construct-barrel', 27: 'total_tracked-construct-barrel', 28: 'total_flickering_tracked-construct-barrel', 29: 'total_detected-construct-sign', 

In [16]:
a = dict.fromkeys(attr2idx.keys(),0)
print(a)

{'video_source': 0, 'cam_num': 0, 'latitude': 0, 'longitude': 0, 'hour_of_day': 0, 'total_detected-bike': 0, 'total_tracked-bike': 0, 'total_flickering_tracked-bike': 0, 'total_detected-bus': 0, 'total_tracked-bus': 0, 'total_flickering_tracked-bus': 0, 'total_detected-car': 0, 'total_tracked-car': 0, 'total_flickering_tracked-car': 0, 'total_detected-construct-equipment': 0, 'total_tracked-construct-equipment': 0, 'total_flickering_tracked-construct-equipment': 0, 'total_detected-construct-cone': 0, 'total_tracked-construct-cone': 0, 'total_flickering_tracked-construct-cone': 0, 'total_detected-construct-pole': 0, 'total_tracked-construct-pole': 0, 'total_flickering_tracked-construct-pole': 0, 'total_detected-construct-post': 0, 'total_tracked-construct-post': 0, 'total_flickering_tracked-construct-post': 0, 'total_detected-construct-barrel': 0, 'total_tracked-construct-barrel': 0, 'total_flickering_tracked-construct-barrel': 0, 'total_detected-construct-sign': 0, 'total_tracked-const

In [36]:
import os
import glob
import datetime
import ast
import json

run_eval = True
if run_eval:
    with open('/home/kuser/Desktop/master_log_features_080519.json', 'w+') as fn:
        json_str = json.dumps([])
        fn.write(json_str)
    
    with open('/home/kuser/Desktop/master_log_features_080519.json', 'r') as fn:
        master_feats = json.load(fn)
            
    for flickers in glob.glob(os.path.join('/media/kuser/Data/','080519_trn','*.json')):
        try:
            with open(flickers, 'r') as fn:
                flicker_data = json.load(fn)
        except Exception as e:
            print("Error occured opening json file {flickers}, skipping")
            continue
            
        if flicker_data and len(flicker_data) > 0 and flicker_data[0].get('feature_vector', None):
            master_feats.append(flicker_data[0]['feature_vector'])
        elif not flicker_data or len(flicker_data) == 0:
            continue
        else:
            print(f'Feature vector not found for: {flickers}, generating...')
            if flicker_data and flicker_data[0].get('videoName', None) and 'unknown' not in flicker_data[0]['videoName'].lower():
                feat_vec = dict.fromkeys(attr2idx.keys(), '0.0')
                if 'cam' in flicker_data[0]['videoName']:
                    cam_num = int(float(flicker_data[0]['videoName'].split('cam')[-1]))
                    feat_vec["cam_num"] = str(cam_num)






                # Generate vector
                feat_vec["video_source"] = flicker_data[0]['videoName']


                if flicker_data[0].get('latitude', None):    
                    feat_vec["latitude"] = str(flicker_data[0]['latitude'])

                if flicker_data[0].get('longitude', None):    
                    feat_vec["longitude"] = str(flicker_data[0]['longitude'])

                if flicker_data[0].get('timestamp', None):
                    timestamp = int(flicker_data[0]['timestamp'])
                    timestamp_s = timestamp/1e9

                    hour = datetime.datetime.fromtimestamp(timestamp_s).strftime('%H')            
                    feat_vec["hour_of_day"] = hour



                for flicker_frame in flicker_data:
                    flicker_frame_detections = []
                    for label in flicker_frame.get('labels'):
                        feat_vec[f"total_{label['category']}"] = str( int( float(feat_vec[f"total_{label['category']}"])) +1 )

                        label_area =  np.abs((label['box2d']['x2'])-(label['box2d']['x1']) * (label['box2d']['y2'])-(label['box2d']['y1']))                
                        flicker_frame_detections.append([f"{label['category']}", label_area, [label["box2d"]["x1"], label["box2d"]["y1"],label["box2d"]["x2"], label["box2d"]["y2"]]] )

                    # Fill in detections
                    for cls in classes:
                        for tracking_type in ['detected', 'tracked', 'flickering_tracked']:

                            class_dets = [det for det in flicker_frame_detections if f'{tracking_type}-{cls}' in det[0]]

                            # Sort class_dets by area size
                            sorted_class_dets = (sorted(class_dets, key = lambda x: x[1]))

                            # Only process 10 object maximum
                            while len(sorted_class_dets) <= 10:
                                sorted_class_dets.append([f'{tracking_type}-{cls}', 0.0, [0.0, 0.0, 0.0, 0.0]])

                            if len(sorted_class_dets) > 10:
                                 sorted_class_dets = sorted_class_dets[:10]

                            # Load the first 10 objects in each category
                            for i, det in enumerate(sorted_class_dets, 1):
                                #x1 
                                feat_vec[f"{tracking_type}-{cls}_{i}_x1"] = str(det[2][0])

                                #y1 
                                feat_vec[f"{tracking_type}-{cls}_{i}_y1"] = str(det[2][1])

                                #x2 
                                feat_vec[f"{tracking_type}-{cls}_{i}_x2"] = str(det[2][2])

                                #y2 
                                feat_vec[f"{tracking_type}-{cls}_{i}_y2"] = str(det[2][3])
                                                         
                flicker_data[0]['feature_vector'] = feat_vec
                master_feats.append(feat_vec)        

                # Save vector in json file
                with open(flickers, 'w+') as fn:
                    json.dump(flicker_data, fn)




                                                     
    # Save vector in json file
    with open('/home/kuser/Desktop/master_log_features_080519.json', 'w+') as fn:
        json_str = json.dumps( master_feats )
        fn.write(json_str)



        

Error occured opening json file {flickers}, skipping
Feature vector not found for: /media/kuser/Data/080519_trn/bagfile_18-09-21_0857_116_canis-majoris_1562714539_bagfile_18-09-21_0857_116_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/bagfile_18-09-21_0857_117_canis-majoris_1562714539_bagfile_18-09-21_0857_117_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/bagfile_18-09-30_1120_29_canis-majoris_1562714539_bagfile_18-09-30_1120_29_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/bagfile_18-09-18_0905_53_canis-majoris_1562714539_bagfile_18-09-18_0905_53_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/bagfile_18-11-20_1935_2_canis-majoris_1562714539_bagfile_18-11-20_1935_2_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/bagfile_18-11-29_1746_3_canis-majoris_1562714539_bagfile_18-11-29_1746_

Feature vector not found for: /media/kuser/Data/080519_trn/alanturing-2019-02-04--11-31-02--cam2_canis-majoris_1562714539_alanturing-2019-02-04--11-31-02--cam2_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/alanturing-2019-04-04--21-49-41--cam1_canis-majoris_1562714539_alanturing-2019-04-04--21-49-41--cam1_flickers.json, generating...
Error occured opening json file {flickers}, skipping
Feature vector not found for: /media/kuser/Data/080519_trn/daisy-2019-05-01--13-45-48--cam0_canis-majoris_1562714539_daisy-2019-05-01--13-45-48--cam0_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/daisy-2019-05-03--00-47-26--cam0_canis-majoris_1562714539_daisy-2019-05-03--00-47-26--cam0_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/alanturing-2019-03-21--16-27-28--cam0_canis-majoris_1562714539_alanturing-2019-03-21--16-27-28--cam0_flickers.json, generating...
Feature vector not fou

Feature vector not found for: /media/kuser/Data/080519_trn/daisy-2019-04-27--15-43-37--cam2_canis-majoris_1562714539_daisy-2019-04-27--15-43-37--cam2_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/chrysippus-2019-03-13--21-07-28--cam1_canis-majoris_1562714539_chrysippus-2019-03-13--21-07-28--cam1_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/chrysippus-2019-03-31--13-53-54--cam1_canis-majoris_1562714539_chrysippus-2019-03-31--13-53-54--cam1_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/chrysippus-2019-04-02--18-22-54--cam1_canis-majoris_1562714539_chrysippus-2019-04-02--18-22-54--cam1_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/alanturing-2019-02-26--14-57-14--cam0_canis-majoris_1562714539_alanturing-2019-02-26--14-57-14--cam0_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/sallyr

Feature vector not found for: /media/kuser/Data/080519_trn/1368-frames_canis-majoris_1562714539_1368-frames_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/1380-frames_canis-majoris_1562714539_1380-frames_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/61-frames_canis-majoris_1562714539_61-frames_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/846-frames_canis-majoris_1562714539_846-frames_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/1079-frames_canis-majoris_1562714539_1079-frames_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/1572-frames_canis-majoris_1562714539_1572-frames_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080519_trn/67-frames_canis-majoris_1562714539_67-frames_flickers.json, generating...
Feature vector not found for: /media/kuser/Data/080

In [37]:
with open('/home/kuser/Desktop/master_log_features_080519.json', 'r') as fn:
    master_feats =  json.load(fn)
    log_source_names = [list(v.values())[0] for idx,v in enumerate(master_feats)] 
    X = np.array([list(v.values())[1:] for idx,v in enumerate(master_feats)], dtype=np.float32)
    
    

In [38]:
print(X) ; print(X.shape); print(log_source_names)

[[  0.        36.135868 -85.504684 ...   0.         0.         0.      ]
 [  0.        36.911636 -81.291565 ...   0.         0.         0.      ]
 [  0.        36.12394  -85.68285  ...   0.         0.         0.      ]
 ...
 [  0.         0.         0.       ...   0.         0.         0.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]
 [  0.         0.         0.       ...   0.         0.         0.      ]]
(9888, 2833)
['bagfile_18-11-16_1757_119', 'bagfile_18-11-16_1757_11', 'bagfile_18-11-16_1757_123', 'bagfile_18-11-16_1757_125', 'bagfile_18-11-16_1757_128', 'bagfile_18-11-16_1757_130', 'bagfile_18-11-16_1757_133', 'bagfile_19-02-13_2056_22', 'bagfile_19-02-13_2056_26', 'bagfile_19-02-13_2056_27', 'bagfile_19-02-13_2056_30', 'bagfile_19-02-13_2056_32', 'bagfile_18-11-18_1454_105', 'bagfile_18-11-18_1454_108', 'bagfile_18-11-18_1454_109', 'bagfile_18-11-18_1454_10', 'bagfile_18-11-18_1454_110', 'bagfile_18-11-18_1454_111', 'bagfile_18-11-18_1454_113'

In [39]:
from sklearn.cluster import MiniBatchKMeans
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
X_scaled = mm_scaler.fit_transform(X)

kmeans = MiniBatchKMeans(n_clusters=30,
                          random_state=0,
                          batch_size=10,
                          max_iter=100).fit(X_scaled)


In [40]:
print(kmeans.cluster_centers_)
print(len(kmeans.cluster_centers_))

[[0.12755103 0.5207079  0.37504616 ... 0.         0.         0.        ]
 [0.21359223 0.3210905  0.58233744 ... 0.         0.         0.        ]
 [0.17938933 0.42542604 0.44550326 ... 0.         0.         0.        ]
 ...
 [0.1590296  0.35724136 0.5395071  ... 0.         0.         0.        ]
 [0.23157896 0.38836592 0.43811482 ... 0.         0.         0.        ]
 [0.17940198 0.35791135 0.5245234  ... 0.         0.         0.        ]]
30


In [41]:
groups = kmeans.predict(X_scaled)

In [42]:
clustered_logs = []

for idx, group in enumerate(groups):
    print(f"log: {log_source_names[idx]} | cluster: {group}")
    clustered_logs.append((log_source_names[idx], group))

log: bagfile_18-11-16_1757_119 | cluster: 1
log: bagfile_18-11-16_1757_11 | cluster: 8
log: bagfile_18-11-16_1757_123 | cluster: 9
log: bagfile_18-11-16_1757_125 | cluster: 14
log: bagfile_18-11-16_1757_128 | cluster: 22
log: bagfile_18-11-16_1757_130 | cluster: 26
log: bagfile_18-11-16_1757_133 | cluster: 0
log: bagfile_19-02-13_2056_22 | cluster: 3
log: bagfile_19-02-13_2056_26 | cluster: 26
log: bagfile_19-02-13_2056_27 | cluster: 16
log: bagfile_19-02-13_2056_30 | cluster: 13
log: bagfile_19-02-13_2056_32 | cluster: 9
log: bagfile_18-11-18_1454_105 | cluster: 9
log: bagfile_18-11-18_1454_108 | cluster: 29
log: bagfile_18-11-18_1454_109 | cluster: 11
log: bagfile_18-11-18_1454_10 | cluster: 8
log: bagfile_18-11-18_1454_110 | cluster: 29
log: bagfile_18-11-18_1454_111 | cluster: 22
log: bagfile_18-11-18_1454_113 | cluster: 8
log: bagfile_18-11-18_1454_114 | cluster: 19
log: bagfile_18-11-18_1454_117 | cluster: 14
log: bagfile_18-11-18_1454_11 | cluster: 6
log: bagfile_18-10-26_1123_1

log: bagfile_18-10-27_1103_161 | cluster: 27
log: bagfile_18-10-27_1103_162 | cluster: 27
log: bagfile_18-10-27_1103_163 | cluster: 14
log: bagfile_18-10-27_1103_164 | cluster: 27
log: bagfile_19-02-20_0029_80 | cluster: 20
log: bagfile_19-02-20_0029_86 | cluster: 27
log: bagfile_19-02-20_0029_87 | cluster: 27
log: bagfile_19-02-20_0029_8 | cluster: 22
log: bagfile_19-02-20_0029_90 | cluster: 10
log: bagfile_19-02-20_0029_91 | cluster: 22
log: bagfile_19-02-20_0029_92 | cluster: 27
log: bagfile_19-02-20_0029_94 | cluster: 27
log: bagfile_19-02-20_0029_95 | cluster: 27
log: bagfile_19-01-11_1741_52 | cluster: 9
log: bagfile_19-01-11_1741_59 | cluster: 9
log: bagfile_19-01-11_1741_5 | cluster: 26
log: bagfile_19-01-11_1741_60 | cluster: 9
log: bagfile_19-01-11_1741_63 | cluster: 9
log: bagfile_19-01-11_1741_66 | cluster: 5
log: bagfile_19-01-11_1741_68 | cluster: 5
log: bagfile_19-01-11_1741_6 | cluster: 26
log: bagfile_19-01-11_1741_70 | cluster: 14
log: bagfile_19-01-11_1741_72 | clust

log: bagfile_19-02-17_0718_19 | cluster: 27
log: bagfile_19-02-17_0718_21 | cluster: 17
log: bagfile_19-02-17_0718_23 | cluster: 8
log: bagfile_19-02-17_0718_24 | cluster: 1
log: bagfile_19-02-17_0718_25 | cluster: 2
log: bagfile_19-02-17_0718_26 | cluster: 6
log: bagfile_19-02-17_0718_3 | cluster: 2
log: bagfile_19-02-17_0718_5 | cluster: 27
log: bagfile_19-02-18_2051_15 | cluster: 22
log: bagfile_19-02-18_2051_37 | cluster: 27
log: bagfile_19-02-18_2316_0 | cluster: 9
log: bagfile_19-02-18_2319_20 | cluster: 27
log: bagfile_19-02-18_2319_7 | cluster: 2
log: bagfile_19-02-19_1118_2 | cluster: 13
log: bagfile_19-02-19_2018_11 | cluster: 26
log: bagfile_19-02-19_2018_37 | cluster: 22
log: bagfile_19-02-19_2150_12 | cluster: 22
log: bagfile_18-11-15_1051_32 | cluster: 6
log: bagfile_18-11-15_1051_35 | cluster: 17
log: bagfile_18-11-15_1051_36 | cluster: 26
log: bagfile_18-11-15_1051_38 | cluster: 6
log: bagfile_18-11-15_1051_39 | cluster: 8
log: bagfile_18-11-15_1051_42 | cluster: 6
log:

log: bagfile_19-02-14_0825_100 | cluster: 19
log: bagfile_19-02-14_0825_103 | cluster: 6
log: bagfile_19-02-14_0825_104 | cluster: 19
log: bagfile_19-02-14_0825_105 | cluster: 6
log: bagfile_19-02-14_0825_106 | cluster: 4
log: bagfile_19-02-14_0825_109 | cluster: 26
log: bagfile_19-02-14_0825_114 | cluster: 9
log: bagfile_19-02-14_0825_115 | cluster: 26
log: bagfile_19-02-14_0825_116 | cluster: 9
log: bagfile_19-02-14_0825_119 | cluster: 26
log: bagfile_19-02-14_0825_120 | cluster: 19
log: bagfile_19-02-14_0825_122 | cluster: 16
log: bagfile_19-02-14_0825_123 | cluster: 19
log: bagfile_19-02-14_0825_126 | cluster: 26
log: bagfile_18-10-27_1103_269 | cluster: 27
log: bagfile_18-10-27_1103_275 | cluster: 27
log: bagfile_18-10-27_1103_276 | cluster: 27
log: bagfile_18-10-27_1103_277 | cluster: 1
log: bagfile_18-10-27_1103_278 | cluster: 1
log: bagfile_18-10-27_1103_27 | cluster: 1
log: bagfile_18-10-27_1103_281 | cluster: 27
log: bagfile_18-10-27_1103_285 | cluster: 27
log: bagfile_18-10-

log: bagfile_18-11-16_1757_68 | cluster: 19
log: bagfile_18-11-16_1757_6 | cluster: 20
log: bagfile_18-11-16_1757_70 | cluster: 4
log: bagfile_18-11-16_1757_73 | cluster: 18
log: bagfile_18-11-16_1757_74 | cluster: 0
log: bagfile_18-11-16_1757_76 | cluster: 5
log: bagfile_19-02-14_0825_80 | cluster: 6
log: bagfile_19-02-14_0825_81 | cluster: 17
log: bagfile_19-02-14_0825_85 | cluster: 0
log: bagfile_19-02-14_0825_86 | cluster: 8
log: bagfile_19-02-14_0825_88 | cluster: 26
log: bagfile_19-02-14_0825_8 | cluster: 0
log: bagfile_19-02-14_0825_95 | cluster: 5
log: bagfile_19-02-14_0825_97 | cluster: 6
log: bagfile_19-02-14_0825_98 | cluster: 8
log: bagfile_19-02-14_0825_9 | cluster: 26
log: bagfile_18-12-04_1848_20 | cluster: 5
log: bagfile_18-12-04_1848_21 | cluster: 26
log: bagfile_18-12-04_1848_23 | cluster: 6
log: bagfile_18-12-04_1848_24 | cluster: 28
log: bagfile_18-12-04_1848_25 | cluster: 3
log: bagfile_18-12-04_1848_27 | cluster: 21
log: bagfile_18-12-04_1848_28 | cluster: 21
log:

...

log: chrysippus-2019-03-05--20-25-13--cam1 | cluster: 3
log: chrysippus-2019-03-04--21-04-16--cam1 | cluster: 20
log: chrysippus-2019-03-05--10-54-34--cam1 | cluster: 15
log: chrysippus-2019-03-05--12-32-23--cam1 | cluster: 19
log: chrysippus-2019-03-05--20-56-13 | cluster: 27
log: chrysippus-2019-03-06--10-51-00--cam1 | cluster: 26
log: chrysippus-2019-03-06--20-36-07--cam1 | cluster: 4
log: chrysippus-2019-03-07--08-31-26--cam2 | cluster: 29
log: chrysippus-2019-03-07--09-04-26--cam0 | cluster: 26
log: chrysippus-2019-03-07--10-01-50--cam1 | cluster: 19
log: chrysippus-2019-03-07--21-19-02--cam1 | cluster: 26
log: chrysippus-2019-03-08--08-12-33--cam1 | cluster: 26
log: chrysippus-2019-03-21--21-22-49--cam1 | cluster: 5
log: chrysippus-2019-03-21--21-28-49--cam1 | cluster: 28
log: chrysippus-2019-03-21--21-37-49--cam1 | cluster: 2
log: chrysippus-2019-03-21--21-37-49--cam2 | cluster: 20
log: chrysippus-2019-03-21--21-51-49--cam1 | cluster: 28
log: chrysippus-2019-03-21--21-53-49--cam

log: alanturing-2019-05-01--23-41-05--cam0 | cluster: 6
log: sallyride-2019-02-07--18-44-37--cam1 | cluster: 21
log: sallyride-2019-02-07--18-54-40--cam0 | cluster: 26
log: sallyride-2019-02-07--19-13-22 | cluster: 22
log: sallyride-2019-02-07--19-20-22--cam1 | cluster: 5
log: sallyride-2019-02-07--19-20-22--cam2 | cluster: 27
log: sallyride-2019-02-07--19-22-22--cam0 | cluster: 22
log: sallyride-2019-02-07--19-32-22--cam0 | cluster: 9
log: sallyride-2019-02-07--23-19-12--cam2 | cluster: 22
log: sallyride-2019-02-07--23-24-12 | cluster: 12
log: sallyride-2019-02-07--23-27-12--cam1 | cluster: 3
log: sallyride-2019-02-07--23-40-40--cam1 | cluster: 11
log: sallyride-2019-02-07--23-40-40--cam2 | cluster: 22
log: alanturing-2019-02-28--18-31-14 | cluster: 28
log: alanturing-2019-02-28--18-33-14--cam0 | cluster: 22
log: alanturing-2019-02-28--18-36-14--cam1 | cluster: 5
log: alanturing-2019-02-28--18-37-14--cam1 | cluster: 26
log: alanturing-2019-02-28--19-20-32--cam0 | cluster: 19
log: alan

log: alanturing-2019-04-02--12-27-52--cam2 | cluster: 22
log: daisy-2019-04-21--21-51-58--cam1 | cluster: 17
log: daisy-2019-04-21--21-54-58--cam2 | cluster: 22
log: daisy-2019-04-21--21-56-58--cam0 | cluster: 11
log: daisy-2019-04-21--22-02-58--cam1 | cluster: 9
log: daisy-2019-04-21--22-14-58 | cluster: 22
log: daisy-2019-04-21--22-54-58--cam1 | cluster: 1
log: daisy-2019-04-21--22-54-58 | cluster: 27
log: lily-2019-04-27--18-27-47--cam0 | cluster: 27
log: lily-2019-04-27--18-44-47--cam1 | cluster: 22
log: lily-2019-04-27--18-52-47--cam0 | cluster: 1
log: lily-2019-04-27--19-01-47 | cluster: 16
log: lily-2019-04-27--19-03-47--cam2 | cluster: 3
log: lily-2019-04-27--19-14-47--cam1 | cluster: 22
log: alanturing-2019-03-02--20-34-32--cam2 | cluster: 29
log: alanturing-2019-03-02--20-39-32--cam0 | cluster: 22
log: alanturing-2019-03-02--20-46-32--cam1 | cluster: 26
log: alanturing-2019-03-02--20-57-32--cam1 | cluster: 3
log: alanturing-2019-03-02--20-58-32--cam1 | cluster: 14
log: alantu

log: chrysippus-2019-04-04--16-52-20--cam2 | cluster: 22
log: chrysippus-2019-04-04--16-56-20--cam0 | cluster: 19
log: lily-2019-04-09--17-46-06 | cluster: 23
log: lily-2019-04-09--17-51-06--cam0 | cluster: 28
log: lily-2019-04-09--17-54-06--cam0 | cluster: 21
log: lily-2019-04-09--18-00-06--cam0 | cluster: 21
log: lily-2019-04-09--18-01-06--cam1 | cluster: 23
log: lily-2019-04-09--18-12-06--cam1 | cluster: 23
log: lily-2019-04-21--20-59-23--cam1 | cluster: 29
log: lily-2019-04-21--21-03-23--cam0 | cluster: 29
log: lily-2019-04-21--21-04-23--cam2 | cluster: 1
log: lily-2019-04-21--21-12-23--cam0 | cluster: 13
log: lily-2019-04-21--21-12-23--cam1 | cluster: 9
log: lily-2019-04-21--21-14-23 | cluster: 29
log: daisy-2019-04-30--17-53-07--cam0 | cluster: 27
log: lily-2019-04-22--23-24-57--cam2 | cluster: 1
log: lily-2019-04-22--23-26-57--cam0 | cluster: 9
log: lily-2019-04-22--23-33-57--cam0 | cluster: 1
log: lily-2019-04-22--23-36-57--cam1 | cluster: 16
log: lily-2019-04-22--23-46-57--cam

log: 1696-frames | cluster: 27
log: 1721-frames | cluster: 14
log: alanturing-2019-02-12--15-07-59--cam1 | cluster: 12
log: alanturing-2019-02-17--10-43-19--cam0 | cluster: 14
log: alanturing-2019-02-21--11-00-02 | cluster: 29
log: alanturing-2019-02-28--09-17-53--cam0 | cluster: 11
log: alanturing-2019-03-02--10-36-23--cam1 | cluster: 5
log: 1461-frames | cluster: 1
log: 1463-frames | cluster: 29
log: 1464-frames | cluster: 22
log: 1465-frames | cluster: 29
log: 1466-frames | cluster: 29
log: 1467-frames | cluster: 29
log: 147-frames | cluster: 14
log: 1471-frames | cluster: 1
log: 1472-frames | cluster: 1
log: 1473-frames | cluster: 1
log: 1475-frames | cluster: 2
log: 1476-frames | cluster: 9
log: 1480-frames | cluster: 22
log: 1482-frames | cluster: 27
log: 1484-frames | cluster: 27
log: 1485-frames | cluster: 22
log: 1490-frames | cluster: 29
log: 1491-frames | cluster: 27
log: 1492-frames | cluster: 11
log: 959-frames | cluster: 4
log: 96-frames | cluster: 8
log: 960-frames | clu

In [43]:
## Workflow for Quasars

## Supernovae Generates Flickers JSON and pushed message to generate feature vector ##

## Download log from flickering db (get all frames under same video_source_name) ##
## Generate Feature Vector ##
## SDave Feature Vector in Cached Json Flickers File ##



In [44]:
def get_best_log(cluster_matrix, category, attr2idx, log_feat_vectors):
    best_log = None
    best_flickering_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_flickering_tracked-{category}'] - 1])
    best_tracked_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1])
    best_detected_cat_element = np.amax(cluster_matrix[:, attr2idx[f'total_detected-{category}'] - 1])
    
    if best_flickering_cat_element > 0:
        print("best_flickering_cat_element", best_flickering_cat_element)
        best_flickering_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_flickering_tracked-{category}'] - 1] == best_flickering_cat_element)[0]

        if type(best_flickering_cat_log_idx) != int:
            if best_flickering_cat_log_idx.any():
                best_flickering_cat_log_idx = best_flickering_cat_log_idx.ravel()[0]
                best_flickering_cat_log_idx = np.asscalar(best_flickering_cat_log_idx)
                
                
                print("best_flickering_cat_log_idx", best_flickering_cat_log_idx)
                best_log_vec = X[best_flickering_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
                
                
#                 best_log = list(log_feat_vectors.keys())[best_flickering_cat_log_idx]
                print("BEST FLICKERING LOG: ", best_log, cluster_matrix[best_flickering_cat_log_idx, attr2idx[f'total_flickering_tracked-{category}'] - 1])
            

    elif best_tracked_cat_element > 0:
        print("best_tracked_cat_element", best_tracked_cat_element)
        best_tracked_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1] == best_tracked_cat_element)[0]

        if type(best_tracked_cat_log_idx) != int:
            if best_tracked_cat_log_idx.any():
                best_tracked_cat_log_idx = best_tracked_cat_log_idx.ravel()[0]
                best_tracked_cat_log_idx = np.asscalar(best_tracked_cat_log_idx)

                print("best_tracked_cat_log_idx", best_tracked_cat_log_idx)
                best_log_vec = X[best_tracked_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
#                 best_log = list(log_feat_vectors.keys())[int(best_tracked_cat_log_idx)]
                print("BEST TRACKED LOG: ", best_log, (cluster_matrix[best_tracked_cat_log_idx, attr2idx[f'total_tracked-{category}'] - 1]))
        
    elif best_detected_cat_element > 0:
        print("best_detected_cat_element", best_detected_cat_element)
        best_detected_cat_log_idx = np.where( cluster_matrix[:, attr2idx[f'total_tracked-{category}'] - 1] == best_detected_cat_element)[0]

        if type(best_detected_cat_log_idx) != int:
            if best_detected_cat_log_idx.any():
                best_detected_cat_log_idx = best_detected_cat_log_idx.ravel()[0]
                best_detected_cat_log_idx = np.asscalar(best_detected_cat_log_idx)
            
                print("best_detected_cat_log_idx", best_detected_cat_log_idx)
                best_log_vec = X[best_detected_cat_log_idx]
                best_log_list = [log for log, vec in log_feat_vectors.items() if np.array_equal(best_log_vec, vec)]
                
                if any(best_log_list):
                    best_log = best_log_list[0]
                
                
#                 best_log = list(log_feat_vectors.keys())[int(best_detected_cat_log_idx)]
                print("BEST DETECTED LOG: ", best_log, (cluster_matrix[best_detected_cat_log_idx, attr2idx[f'total_tracked-{category}'] - 1]))
        
    else:
         print(f"No Best log found for category: {category}.. Skipping")
            
          
    return best_log

In [45]:
from collections import OrderedDict 

log_cluster = defaultdict(list)

for log in clustered_logs:
    log_cluster[log[1]].append(log[0])
    
logs_selected = []
labels  = [ "bike", "bus", "car", "construct-equipment", "construct-pole", "construct-post", "construct-sign", "motorcycle", "person", "rider", "traffic light", "traffic light-amber", "traffic light-green", "traffic light-red", "traffic sign", "traffic sign-slow_sign", "traffic sign-speed_sign", "traffic sign-stop_sign", "trailer", "train", "truck",]
with open('/home/kuser/Desktop/master_log_features_080519.json', 'r') as fn:
    master_feats =  json.load(fn)
        
for cluster, logs in log_cluster.items():
    # Generate table of logs and feature vectors for each cluster
    log_feat_vectors = OrderedDict()
    
    # sort logs by most flickers of category
    # select log with most
    print("\n\nCluster:", cluster)
    
    for log in sorted(logs):
#         with open(glob.glob(os.path.join(f"/home/dean/Desktop/flickers/{log}*"))[0], 'r') as fn:
#             flicker_data = json.loads(fn.read())
        
            
        log_idx = [idx for idx, feat in enumerate(master_feats) if feat['video_source'] == log][0]
        log_feat_vectors[log] = np.array(list(master_feats[log_idx].values())[1:], dtype= np.float32)
        

    daytime_delta = [9, 10, 11, 12, 13, 14, 15, 16, 17 ,18]
    for cat in labels:
        # Get best daytime log for category
        X = np.array([v for k, v in log_feat_vectors.items() if k not in logs_selected and v[attr2idx['hour_of_day']] in daytime_delta])
        if X.any():
            best_log = get_best_log(X, cat, attr2idx, log_feat_vectors)
            if best_log: 
                logs_selected.append(best_log)
                
        # Get best nighttime log for category
        X = np.array([v for k, v in log_feat_vectors.items() if k not in logs_selected and v[attr2idx['hour_of_day']] not in daytime_delta])
        if X.any():
            best_log = get_best_log(X, cat, attr2idx, log_feat_vectors)
            if best_log: 
                logs_selected.append(best_log)
    
print(set(logs_selected))   
print(len(set(logs_selected)))



Cluster: 1
best_flickering_cat_element 11.0
best_flickering_cat_log_idx 1
BEST FLICKERING LOG:  bagfile_18-10-27_1103_7 11.0
best_flickering_cat_element 224.0
best_flickering_cat_log_idx 284
BEST FLICKERING LOG:  chrysippus-2019-03-29--09-21-16--cam0 224.0
best_flickering_cat_element 90.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  daisy-2019-06-03--12-19-20--cam1 90.0
best_flickering_cat_element 1528.0
best_flickering_cat_log_idx 138
BEST FLICKERING LOG:  bagfile_18-10-10_0801_8 1528.0
best_flickering_cat_element 516.0
best_flickering_cat_log_idx 4
BEST FLICKERING LOG:  lily-2019-04-21--10-46-26--cam0 516.0
best_flickering_cat_element 8561.0
best_flickering_cat_log_idx 221
BEST FLICKERING LOG:  bagfile_19-02-14_0812_12 8561.0
best_flickering_cat_element 5.0
best_flickering_cat_log_idx 1
BEST FLICKERING LOG:  bagfile_19-02-16_1328_1_ 5.0
best_flickering_cat_element 294.0
best_flickering_cat_log_idx 142
BEST FLICKERING LOG:  bagfile_18-10-26_1120_0 294.0
best_flickering_cat_el

BEST FLICKERING LOG:  lily-2019-04-12--16-52-50--cam0 954.0
best_flickering_cat_element 1149.0
best_flickering_cat_log_idx 8
BEST FLICKERING LOG:  chrysippus-2019-03-25--20-27-09--cam0 1149.0
best_flickering_cat_element 15110.0
best_flickering_cat_log_idx 176
BEST FLICKERING LOG:  bagfile_18-10-10_0801_217 15110.0
best_flickering_cat_element 6.0
best_flickering_cat_log_idx 5
BEST FLICKERING LOG:  bagfile_19-01-08_1341_34 6.0
best_flickering_cat_element 234.0
best_flickering_cat_log_idx 421
BEST FLICKERING LOG:  chrysippus-2019-03-20--09-04-37--cam0 234.0
best_flickering_cat_element 307.0
best_flickering_cat_element 762.0
best_flickering_cat_log_idx 273
BEST FLICKERING LOG:  bagfile_19-01-11_1836_9 762.0
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-post.. Skipping
best_flickering_cat_element 25.0
best_flickering_cat_log_idx 8
BEST FLICKERING LOG:  lily-2019-05-02--17-44-54--cam0 25.0
best_flickering_cat_element 64.0
best_flickering_

best_flickering_cat_element 88.0
best_flickering_cat_log_idx 187
BEST FLICKERING LOG:  bagfile_18-10-17_1001_4 88.0
best_flickering_cat_element 238.0
best_flickering_cat_log_idx 6
BEST FLICKERING LOG:  alanturing-2019-03-07--15-15-31--cam1 238.0
best_flickering_cat_element 4664.0
best_flickering_cat_log_idx 329
BEST FLICKERING LOG:  bagfile_19-02-18_1419_24 4664.0
best_tracked_cat_element 18.0
best_tracked_cat_log_idx 5
BEST TRACKED LOG:  alanturing-2019-02-28--18-33-14--cam0 18.0
best_flickering_cat_element 24.0
best_flickering_cat_log_idx 3
BEST FLICKERING LOG:  1392-frames 24.0
best_flickering_cat_element 19.0
best_flickering_cat_log_idx 6
BEST FLICKERING LOG:  alanturing-2019-04-09--16-53-14--cam0 19.0
best_flickering_cat_element 422.0
best_flickering_cat_log_idx 224
BEST FLICKERING LOG:  bagfile_18-11-27_1303_16 422.0
best_tracked_cat_element 105.0
best_tracked_cat_log_idx 12
BEST TRACKED LOG:  chrysippus-2019-04-01--07-29-32--cam2 105.0
best_flickering_cat_element 22.0
best_flick

best_flickering_cat_element 26.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  alanturing-2019-02-28--09-36-53--cam0 26.0
best_flickering_cat_element 269.0
best_flickering_cat_log_idx 19
BEST FLICKERING LOG:  alanturing-2019-03-25--11-19-45--cam1 269.0
best_flickering_cat_element 24.0
best_flickering_cat_log_idx 3
BEST FLICKERING LOG:  lily-2019-04-28--16-30-29--cam1 24.0
best_flickering_cat_element 212.0
best_flickering_cat_log_idx 156
BEST FLICKERING LOG:  bagfile_19-02-17_1556_15 212.0
best_tracked_cat_element 6.0
best_tracked_cat_log_idx 2
BEST TRACKED LOG:  bagfile_18-09-30_1120_7 6.0
best_flickering_cat_element 77.0
best_tracked_cat_element 1.0
best_flickering_cat_element 711.0
No Best log found for category: traffic light-amber.. Skipping
best_flickering_cat_element 13.0
best_flickering_cat_log_idx 155
BEST FLICKERING LOG:  bagfile_19-02-17_1231_0 13.0
best_tracked_cat_element 1.0
best_tracked_cat_log_idx 1
BEST TRACKED LOG:  1949-frames 1.0
best_flickering_cat_element 50.

BEST FLICKERING LOG:  lily-2019-05-07--18-59-30--cam2 306.0
best_flickering_cat_element 560.0
best_flickering_cat_element 5373.0
best_flickering_cat_log_idx 157
BEST FLICKERING LOG:  bagfile_19-02-16_0112_29 5373.0
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-slow_sign.. Skipping
best_tracked_cat_element 6.0
best_flickering_cat_element 124.0
best_flickering_cat_log_idx 9
BEST FLICKERING LOG:  1393-frames 124.0
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: trailer.. Skipping
best_flickering_cat_element 70.0
best_flickering_cat_log_idx 80
BEST FLICKERING LOG:  alanturing-2019-04-17--08-25-34--cam1 70.0
No Best log found for category: train.. Skipping
No Best log found for category: train.. Skipping
best_flickering_cat_element 139.0
best_flickering_cat_element 2746.0
best_flickering_cat_log_idx 83
BEST FLICK

...

best_flickering_cat_element 20.0
best_flickering_cat_log_idx 1
BEST FLICKERING LOG:  bagfile_18-11-03_1323_0 20.0
best_flickering_cat_element 95.0
best_flickering_cat_log_idx 96
BEST FLICKERING LOG:  chrysippus-2019-05-22--11-36-33--cam1 95.0
best_flickering_cat_element 127.0
best_flickering_cat_log_idx 1
BEST FLICKERING LOG:  bagfile_19-01-10_0744_30 127.0
best_flickering_cat_element 1228.0
best_flickering_cat_log_idx 18
BEST FLICKERING LOG:  459-frames 1228.0
No Best log found for category: construct-post.. Skipping
No Best log found for category: construct-post.. Skipping
best_tracked_cat_element 3.0
best_tracked_cat_log_idx 2
BEST TRACKED LOG:  chrysippus-2019-04-03--18-03-26 3.0
best_flickering_cat_element 542.0
best_flickering_cat_log_idx 46
BEST FLICKERING LOG:  bagfile_18-09-20_1417_0 542.0
best_flickering_cat_element 97.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  chrysippus-2019-05-22--11-37-33--cam1 97.0
best_flickering_cat_element 1104.0
best_flickering_cat_log_idx

best_flickering_cat_element 70.0
best_flickering_cat_log_idx 11
BEST FLICKERING LOG:  daisy-2019-05-01--10-10-06--cam1 70.0
best_flickering_cat_element 230.0
best_flickering_cat_log_idx 418
BEST FLICKERING LOG:  bagfile_18-11-18_1524_6 230.0
best_flickering_cat_element 15.0
best_flickering_cat_log_idx 3
BEST FLICKERING LOG:  bagfile_18-10-10_0710_0 15.0
best_flickering_cat_element 2195.0
best_flickering_cat_log_idx 302
BEST FLICKERING LOG:  bagfile_18-10-10_0801_95 2195.0
best_flickering_cat_element 655.0
best_flickering_cat_log_idx 3
BEST FLICKERING LOG:  bagfile_18-12-24_2030_11 655.0
best_flickering_cat_element 6889.0
best_flickering_cat_log_idx 450
BEST FLICKERING LOG:  bagfile_19-02-14_1145_102 6889.0
best_flickering_cat_element 12.0
best_flickering_cat_log_idx 3
BEST FLICKERING LOG:  bagfile_18-12-25_1054_4 12.0
best_flickering_cat_element 705.0
best_flickering_cat_log_idx 429
BEST FLICKERING LOG:  bagfile_19-01-08_1015_16 705.0
best_flickering_cat_element 28.0
best_flickering_ca

best_flickering_cat_element 45.0
best_flickering_cat_element 694.0
best_flickering_cat_log_idx 47
BEST FLICKERING LOG:  bagfile_18-11-15_1051_100 694.0
best_flickering_cat_element 130.0
best_flickering_cat_element 2593.0
best_flickering_cat_log_idx 48
BEST FLICKERING LOG:  bagfile_18-11-15_1051_5 2593.0
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-slow_sign.. Skipping
No Best log found for category: traffic sign-speed_sign.. Skipping
best_flickering_cat_element 162.0
best_flickering_cat_log_idx 110
BEST FLICKERING LOG:  sallyride-2019-03-29--17-19-16--cam1 162.0
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
best_flickering_cat_element 2.0
best_flickering_cat_element 1073.0
best_flickering_cat_log_idx 49
BEST FLICKERING LOG:  bagfile_18-11-15_2106_9 1073.0
No Best log found for category: train.. Skipping
No Best log found for category: t

best_flickering_cat_element 1540.0
best_flickering_cat_log_idx 129
BEST FLICKERING LOG:  daisy-2019-04-22--18-51-43--cam1 1540.0
No Best log found for category: train.. Skipping
No Best log found for category: train.. Skipping
best_flickering_cat_element 546.0
best_flickering_cat_element 5837.0
best_flickering_cat_log_idx 100
BEST FLICKERING LOG:  bagfile_18-12-26_1729_5 5837.0


Cluster: 7
best_flickering_cat_element 14.0
best_flickering_cat_log_idx 2
BEST FLICKERING LOG:  bagfile_18-10-26_1120_45 14.0
best_flickering_cat_element 29.0
best_flickering_cat_log_idx 42
BEST FLICKERING LOG:  bagfile_18-10-26_1120_8 29.0
best_flickering_cat_element 15.0
best_flickering_cat_log_idx 1
BEST FLICKERING LOG:  bagfile_18-10-26_1120_35 15.0
best_flickering_cat_element 860.0
best_flickering_cat_log_idx 12
BEST FLICKERING LOG:  bagfile_18-10-10_0801_5 860.0
best_flickering_cat_element 2530.0
best_flickering_cat_element 4301.0
best_flickering_cat_log_idx 37
BEST FLICKERING LOG:  bagfile_18-10-26_1120

best_flickering_cat_element 15.0
best_flickering_cat_element 9.0
best_flickering_cat_log_idx 40
BEST FLICKERING LOG:  bagfile_19-01-09_1919_8 9.0
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: traffic sign-stop_sign.. Skipping
No Best log found for category: trailer.. Skipping
best_flickering_cat_element 61.0
best_flickering_cat_log_idx 10
BEST FLICKERING LOG:  alanturing-2019-04-03--13-23-42--cam2 61.0
No Best log found for category: train.. Skipping
No Best log found for category: train.. Skipping
best_flickering_cat_element 790.0
best_flickering_cat_element 1750.0
best_flickering_cat_log_idx 24
BEST FLICKERING LOG:  bagfile_18-11-15_1051_44 1750.0


Cluster: 25
best_flickering_cat_element 2557.0
best_flickering_cat_log_idx 54
BEST FLICKERING LOG:  892-frames 2557.0
best_flickering_cat_element 34.0
best_flickering_cat_element 1247.0
best_flickering_cat_element 317.0
best_flickering_cat_log_idx 21
BEST FLICKERING LOG:  391-frames 317.

In [46]:
def update_traffic_light_cats(data):
    for img_data in data:
        if img_data.get('labels', None):
            for label in [l for l in  img_data['labels'] if l['category'] == 'traffic light' and l.get('attributes', None)]:
                if label['attributes'].get('trafficLightColor', None):
                    if label['attributes']['trafficLightColor'][1].lower() == 'green' or label['attributes']['trafficLightColor'][1].lower() == 'g'  or label['attributes']['trafficLightColor'][0] == 1:
                        label['category'] = 'traffic light-green'
                    elif label['attributes']['trafficLightColor'][1].lower() == 'yellow' or label['attributes']['trafficLightColor'][1].lower() == 'y'  or label['attributes']['trafficLightColor'][0] == 2:
                        label['category'] = 'traffic light-amber'
                    elif label['attributes']['trafficLightColor'][1].lower() == 'red' or label['attributes']['trafficLightColor'][1].lower() == 'r'  or label['attributes']['trafficLightColor'][0] == 3:
                        label['category'] = 'traffic light-red'
                    else:
                        label['category'] = 'traffic light'

                # Support both old and new bdd formats
                elif label['attributes'].get('Traffic Light Color', None):
                    if label['attributes']['Traffic Light Color'][1].lower() == 'g' or label['attributes']['Traffic Light Color'][0] == 1:
                        label['category'] = 'traffic light-green'
                    elif label['attributes']['Traffic Light Color'][1].lower() == 'y' or label['attributes']['Traffic Light Color'][0] == 2:
                        label['category'] = 'traffic light-amber'
                    elif label['attributes']['Traffic Light Color'][1].lower() == 'r' or label['attributes']['Traffic Light Color'][0] == 3:
                        label['category'] = 'traffic light-red'
                    else:
                        label['category'] = 'traffic light'
    return data

In [47]:
check_s3 = True
def send_to_s3( img_path, s3_bucket = 'kache-scalabel/kache_ai/frames/'):
    s3_path = os.path.join(s3_bucket, path_leaf(img_path))
    s3_bucket = 's3://'+s3_bucket
    sp = subprocess.Popen("aws s3 cp {} {}".format(img_path, s3_bucket), shell=True, stdout=PIPE)
    out_str = sp.communicate()
    return os.path.join('https://s3-us-west-2.amazonaws.com', s3_path)

In [48]:
anns_dir = '/media/kuser/Data/081119_trn/'
os.makedirs(anns_dir, exist_ok=True)
print(set(logs_selected))   
get_dataset(video_source_names = list(set(logs_selected)), kache_anns_dir = anns_dir)

{'290-frames', 'alanturing-2019-02-06--14-45-00--cam1', 'alanturing-2019-03-25--11-19-45--cam1', '414-frames', '1042-frames', 'bagfile_19-01-11_1836_1', 'sallyride-2019-03-26--14-31-53--cam1', 'chrysippus-2019-03-29--09-21-16--cam0', 'bagfile_19-01-04_1717_2', 'daisy-2019-04-30--13-00-59--cam2', 'bagfile_18-12-13_1933_4', 'daisy-2019-05-28--15-55-11--cam2', '415-frames', 'alanturing-2019-04-03--13-23-42--cam2', 'chrysippus-2019-03-26--19-09-33--cam2', 'sallyride-2019-04-17--10-00-31--cam1', 'bagfile_18-11-15_1051_8', 'bagfile_18-11-27_0609_13', 'bagfile_18-09-19_1016_17', '380-frames', 'bagfile_19-02-18_1213_0', 'daisy-2019-04-28--11-58-22--cam1', 'chrysippus-2019-01-14--17-45-55', 'bagfile_18-09-21_0857_70', 'sallyride-2019-03-05--23-39-49--cam1', 'bagfile_18-09-21_0857_185', 'bagfile_18-09-22_0828_135', 'daisy-2019-05-01--19-16-30--cam0', 'bagfile_18-11-03_1047_0', '1041-frames', 'chrysippus-2018-12-12--11-52-41--cam1', 'bagfile_18-10-10_0801_249', 'daisy-2019-05-01--11-42-06--cam2',

NameError: name 'get_dataset' is not defined

In [51]:
### Filter anns dir. Download logs ###
with open('logs_selected.txt', 'w') as f:
    f.write(str(list(set(logs_selected))))

In [ ]:
import random
import glob
import json
import numpy as np
import hashlib
import toml
import shutil
import subprocess
from subprocess import Popen,PIPE,STDOUT,call


dataset_files = glob.glob(os.path.join(filtered_anns_dir, '*.json'))
dataset_stats = {}

for idx, ann_file in enumerate(dataset_files):
    print(ann_file)
    with open(ann_file, 'r') as f:
        dataset = json.load(f)
        dataset_stats[idx] = len(dataset)
        
print(dataset_stats)
tot_frames = np.sum(list(dataset_stats.values()))
print(tot_frames)

remove_count = tot_frames - 30000 # labeling budget (30k in production)

if remove_count > 0:
    databatch_portions = [float(datalen/tot_frames) for datalen in dataset_stats.values()]
    remove_counts = [int(remove_count*portion)+1 for portion in databatch_portions]
    
    # Randomly remove frames based on remove count
    for idx, ann_file in enumerate(dataset_files):
        with open(ann_file, 'r') as f:
            dataset = json.load(f)
        frames = list(range(len(dataset)))
        random.shuffle(frames)
        deleted_frames = frames[:remove_counts[idx]]
        dataset = [img_data for idx, img_data in enumerate(dataset) if  idx not in deleted_frames]
        with open(ann_file, 'w+') as f:
            json.dump(dataset, f)
            

# Change name/url to s3_url
for idx, ann_file in enumerate(dataset_files):
    
    with open(ann_file, 'r') as f:
        dataset = json.load(f)

    for img_data in dataset:
        hex_val = os.path.abspath(img_data['url']).replace('/', '^^')
        hash_object = hashlib.sha1(str.encode(hex_val))
        hex_dig = hash_object.hexdigest()

        tmp_img_dir = os.path.join(anns_dir, 'images')
        os.makedirs(tmp_img_dir, exist_ok = True)
        orig_frame_path = img_data['url']
        with open('/media/kuser/Data/nebula/pipelines/configs/entanglement_cfg.tml', 'r') as tmlfile:
            cfg_data = toml.load(tmlfile)

        orig_frame_path = os.path.join(cfg_data['entanglement']['master_nebula_dir'], img_data['url'].split('nebula')[-1].strip('/') )
        new_path = os.path.join(tmp_img_dir, str(hex_dig)+os.path.splitext(path_leaf(img_data['url']))[-1])

        shutil.copyfile(orig_frame_path, new_path)
        s3uri = send_to_s3( new_path )

        img_data['url'] = s3uri



    with open(ann_file, 'w+') as f:
        json.dump(dataset, f)
    
    